In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk

# from d2l import torch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset('lansinuote/ChnSentiCorp', keep_in_memory=True)
        dataset = load_from_disk('./data/ChnSentiCorp')
        
        print(dataset)
        def f(data):
            return len(data['text']) > 30
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text

In [4]:
dataset = Dataset('train')
len(dataset) # , dataset[0]

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/validation/cache-cf45964edee402a8.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c6f7400aef16ddba.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-478819d08c52879a.arrow


DatasetDict({
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
})


3

In [5]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def collate_fn(data):
    
    # print(data)
    # print('data长度: ', len(data))
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    # print('编码后的data打印: ', data.keys())
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # 把第15个词固定替换为mask
    # print('第15个词: %s' % input_ids[:, 15])
    
    # 这里直接使用了编码后的数据作为真实预测值
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    # print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [12]:
# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True
                                    )


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    print(token.decode(labels[0]))
    print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    print("")
    
    if i >= 5:
        break

574
[CLS] 这 本 书 我 又 反 复 的 读 了 几 遍 ， 作 [MASK] 的 语 句 虽 然 没 有 别 人 太 好 ， 但 [SEP]
者
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 真 没 想 到 我 每 次 在 当 当 网 上 精 心 [MASK] 儿 子 挑 选 的 图 书, 他 都 非 常 的 [SEP]
为
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 我 非 常 不 满 意 新 月 阁 客 栈 ， 如 果 [MASK] 零 分 可 以 选 ， 我 宁 可 一 分 也 不 [SEP]
有
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 很 漂 亮 ， 黑 漆 加 白 灯 很 亮 ， 配 置 [MASK] 经 很 高 了 ， 价 格 能 接 受 ， 一 直 [SEP]
已
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 国 庆 期 间 住 过 本 酒 店 ， 我 的 评 价 [MASK] 是 " 差, 实 在 实 际 太 差 ". 第 [SEP]
就
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 看 完 不 知 道 作 者 到 底 想 说 什 么 ， [MASK] 了 解 一 下 具 体 的 方 法 结 果 一 无 [SEP]
想
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])



## 加载预训练模型

In [8]:

# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)


def test_1():
    # 模型试算
    out = pretrained(input_ids=input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids
                    )

    out.last_hidden_state.shape
    pass

## 定义下游任务模型

In [13]:

# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        
        # pretrained = pretrained.to(device)

        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        
        out = self.decoder(out.last_hidden_state[:, 15])
        return out


model = Model()


#model(input_ids=input_ids,
#      attention_mask=attention_mask,
#      token_type_ids=token_type_ids
#     ).shape


# 模型转移到GPU上
model.to(device)

Model(
  (decoder): Linear(in_features=768, out_features=21128, bias=True)
)

In [19]:
list(model.parameters())[0].device

device(type='cuda', index=0)

## 训练

In [14]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)

# 损失函数
loss = torch.nn.CrossEntropyLoss()


model.train()


print("training on ", device)


for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        # print('labels', labels)
        # labels tensor([2523, 1962, ....,  6817, 1962])
        
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)
        
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids
                   )
        l = loss(out, labels)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        if i % 200 == 0:
            out = out.cpu()
            labels = labels.cpu()
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print('训练批次: %s 当前第 %s 轮训练 loss: %s acc: %s' % \
                  (epoch, i, l.item(), accuracy))

training on  cuda
训练批次: 0 当前第 0 轮训练 loss: 10.269051551818848 acc: 0.0
训练批次: 0 当前第 100 轮训练 loss: 6.521791458129883 acc: 0.125
训练批次: 0 当前第 200 轮训练 loss: 5.197349548339844 acc: 0.25
训练批次: 0 当前第 300 轮训练 loss: 5.080999851226807 acc: 0.25
训练批次: 0 当前第 400 轮训练 loss: 3.2679288387298584 acc: 0.5625
训练批次: 0 当前第 500 轮训练 loss: 3.4124574661254883 acc: 0.5
训练批次: 1 当前第 0 轮训练 loss: 2.7553489208221436 acc: 0.5
训练批次: 1 当前第 100 轮训练 loss: 3.156139612197876 acc: 0.5625
训练批次: 1 当前第 200 轮训练 loss: 2.1555659770965576 acc: 0.625
训练批次: 1 当前第 300 轮训练 loss: 1.5703136920928955 acc: 0.8125
训练批次: 1 当前第 400 轮训练 loss: 1.6401739120483398 acc: 0.75
训练批次: 1 当前第 500 轮训练 loss: 2.5194320678710938 acc: 0.625
训练批次: 2 当前第 0 轮训练 loss: 1.0090479850769043 acc: 0.9375
训练批次: 2 当前第 100 轮训练 loss: 1.579595685005188 acc: 0.625
训练批次: 2 当前第 200 轮训练 loss: 0.6561413407325745 acc: 0.875
训练批次: 2 当前第 300 轮训练 loss: 1.3835166692733765 acc: 0.8125
训练批次: 2 当前第 400 轮训练 loss: 1.1443946361541748 acc: 0.75
训练批次: 2 当前第 500 轮训练 loss: 1.8488258123397827 a

## 模型保存

In [15]:
# 保存

model_save_path = 'chinese_full_vacant_mission_2023_4_10.pt'
# torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth


In [ ]:

# 加载
# 加载保存的模型
model = Model()
model.load_state_dict(torch.load(model_save_path))

In [19]:
#测试
def test_calculate():
    
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset['test'],
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        if i == 15:
            break

        with torch.no_grad():
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.argmax(dim=1).cpu()
        labels = labels.cpu()
        
        correct += (out == labels).sum().item()
        total += len(labels)

        print('序: %s 输入内容: %s' % (i, token.decode(input_ids[0])))
        print('解码: ', token.decode(labels[0]))
        
    print('acc: %.2f' % (correct / total))
    pass

In [20]:
# 预测
test_calculate()

序: 0 输入内容: [CLS] 很 满 意 的 一 家 酒 店 ， 装 修 不 错 ， [MASK] 天 住 下 晚 上 房 间 的 网 络 出 了 问 [SEP]
解码:  那
序: 1 输入内容: [CLS] 酒 店 位 置 很 偏 ， 大 厅 很 漂 亮 ， 可 [MASK] 房 间 里 有 刚 装 修 的 味 道 ， 房 间 [SEP]
解码:  是
序: 2 输入内容: [CLS] 风 扇 确 实 够 响 的 ， 尤 其 是 到 晚 上 [MASK] 围 安 静 下 来 。 风 扇 频 频 开 启 ， [SEP]
解码:  周
序: 3 输入内容: [CLS] 没 有 许 多 网 友 评 价 热 度 高 的 问 题 [MASK] 第 一 次 要 手 动 安 装 winxp ， 主 板 [SEP]
解码:  ，
序: 4 输入内容: [CLS] 先 买 了 《 不 一 样 的 卡 梅 拉 》 的 前 [MASK] 册 给 儿 子 ， 每 天 晚 上 睡 觉 前 都 [SEP]
解码:  六
序: 5 输入内容: [CLS] 【 荐 书 】 专 家 认 为 目 前 的 由 次 凭 [MASK] 机 引 发 的 金 融 海 啸 只 是 开 始 ， [SEP]
解码:  危
序: 6 输入内容: [CLS] 当 时 优 惠 了 1300 ， 配 置 高 ， 相 对 于 [MASK] 购 价 来 说 性 价 比 太 高 了 。 不 过 [SEP]
解码:  抢
序: 7 输入内容: [CLS] 我 的 订 单 是 7 月 17 日 就 发 出 来 了 [MASK] 但 是 现 在 是 8 月 2 日 ， 我 还 没 [SEP]
解码:  ，
序: 8 输入内容: [CLS] 刚 看 到 第 一 本 的 名 字 《 我 想 去 看 [MASK] 》 就 觉 得 很 可 爱 ， 然 后 看 内 容 [SEP]
解码:  海
序: 9 输入内容: [CLS] 酒 店 的 位 置 很 好, 距 离 火 车 站 非 [MASK] 近. 总 提 感 觉 酒 店 的 性 价 比 不 [SEP]
解码:  常
序: 10 输入内容: [CLS] 我 家 小 宝 同 学 特 别 喜 爱 卡 梅 拉 系 [MASK] 故 事, 每 天 晚 上 睡 觉 前 非 讲 不 [SEP]
解码